##07. 행렬 분해를 이용한 잠재 요인 협업 필터링 실습

- 일반적으로 행렬 분해 에는 SVD가 자주 사용되지만 사용자-아이템 평점 행렬에는 사용자가 평점을 매기지 않은 널 데이터가 많음 -> 주로 SGD나 ALS기반의 행렬분해를 이용함
- 이번에는 SGD 기반의 행렬 분해를 구현하고 이를 기반으로 사용자에게 영화를 추천하는 실습을 구현해보고자 함

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.metrics import mean_squared_error

def get_rmse(R,P,Q,non_zeros):
  error=0
  # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
  full_pred_metrix = np.dot(P,Q.T)

  # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스를 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
  x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
  y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
  R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
  full_pred_matrix_non_zeros = full_pred_metrix[x_non_zero_ind, y_non_zero_ind]
  mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
  rmse = np.sqrt(mse)

  return rmse

In [3]:
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
  num_users, num_items = R.shape
  #P와 Q 매트릭스의 크기를 지정하고 정규 분포를 가진 랜덤한 값으로 입력합니다.
  np.random.seed(1)
  P=np.random.normal(scale=1./K, size=(num_users, K))
  Q=np.random.normal(scale=1./K, size=(num_items, K))

  # R>0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장.
  non_zeros = [(i,j, R[i,j]) for i in range(num_users) for j in range(num_items) if R[i, j]>0]

  # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트.
  for step in range(steps):
    for i,j, r in non_zeros:
      # 실제 값과 예측 값의 차이인 오류 값 구함
      eij = r-np.dot(P[i,:], Q[j,:].T)
      # Regularization을 반영한 SGD 업데이트 공식 적용
      P[i,:] = P[i,:] + learning_rate*(eij*Q[j,:] - r_lambda*P[i,:])
      Q[j,:] = Q[j,:] + learning_rate*(eij*P[i,:] - r_lambda*Q[j,:])

    rmse = get_rmse(R, P, Q, non_zeros)
    if(step % 10) ==0:
      print("### iteration step: ",step, "rmse:", rmse)

  return P, Q

In [4]:
# 영화 평점 행렬 데이터를 새롭게 DataFrame으로 로딩 후 다시 사용자-아이템 평점 행렬로 생성
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-latest-small/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-latest-small/ratings.csv')
ratings = ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns = 'movieId')

# title 칼럼을 얻기 위해 movies와 조인 수행
rating_movies = pd.merge(ratings, movies, on='movieId')

# columns='title'로 title 칼럼으로 pivot 수행.
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

Mounted at /content/drive


In [5]:
P, Q = matrix_factorization(ratings_matrix.values, K=50, steps=200, learning_rate=0.01, r_lambda=0.01)
pred_matrix = np.dot(P,Q.T)

### iteration step:  0 rmse: 2.9023619751336867
### iteration step:  10 rmse: 0.7335768591017927
### iteration step:  20 rmse: 0.5115539026853442
### iteration step:  30 rmse: 0.37261628282537446
### iteration step:  40 rmse: 0.2960818299181014
### iteration step:  50 rmse: 0.2520353192341642
### iteration step:  60 rmse: 0.22487503275269854
### iteration step:  70 rmse: 0.2068545530233154
### iteration step:  80 rmse: 0.19413418783028685
### iteration step:  90 rmse: 0.18470082002720406
### iteration step:  100 rmse: 0.17742927527209104
### iteration step:  110 rmse: 0.1716522696470749
### iteration step:  120 rmse: 0.16695181946871726
### iteration step:  130 rmse: 0.16305292191997542
### iteration step:  140 rmse: 0.15976691929679646
### iteration step:  150 rmse: 0.1569598699945732
### iteration step:  160 rmse: 0.15453398186715425
### iteration step:  170 rmse: 0.15241618551077643
### iteration step:  180 rmse: 0.1505508073962831
### iteration step:  190 rmse: 0.1488947091323209


In [6]:
# 반환된 예측 사용자-아이템 평점 행렬을 영화 타이틀을 칼럼명으로 가지는 데이터프레임으로 변경
ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index=ratings_matrix.index, columns=ratings_matrix.columns)
ratings_pred_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,3.055084,4.092018,3.564130,4.502167,3.981215,1.271694,3.603274,2.333266,5.091749,3.972454,...,1.402608,4.208382,3.705957,2.720514,2.787331,3.475076,3.253458,2.161087,4.010495,0.859474
2,3.170119,3.657992,3.308707,4.166521,4.311890,1.275469,4.237972,1.900366,3.392859,3.647421,...,0.973811,3.528264,3.361532,2.672535,2.404456,4.232789,2.911602,1.634576,4.135735,0.725684
3,2.307073,1.658853,1.443538,2.208859,2.229486,0.780760,1.997043,0.924908,2.970700,2.551446,...,0.520354,1.709494,2.281596,1.782833,1.635173,1.323276,2.887580,1.042618,2.293890,0.396941


In [7]:
def get_unseen_movies(ratings_matrix, userId):
  #userId로 입력받은 사용자의 모든 영화 정보를 추출해 Series로 반환함.
  # 반환된 user_rating은 영화명(title)을 인덱스로 가지는 Series 객체임.
  user_rating = ratings_matrix.loc[userId, :]

  # user_rating이 0보다 크면 기존에 관람한 영화임. 대상 인덱스를 추출해 list 객체로 만듦.
  already_seen = user_rating[user_rating >0].index.tolist()

  # 모든 영화명을 list 객체로 만듦.
  movies_list = ratings_matrix.columns.tolist()

  # list comprehension 으로 already_seen에 해당하는 영화는 movies_list에서 제외함.
  unseen_list = [movie for movie in movies_list if movie not in already_seen]

  return unseen_list

In [8]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
  # 예측 평점 DataFrame에서 사용자 id 인덱스와 unseen_list로 들어온 영화명 칼럼을 추출해
  # 가장 예측 평점이 높은 순으로 정렬함.
  recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[:top_n]
  return recomm_movies

In [9]:
# 사용자가 관람하지 않은 영화명 추출
unseen_list = get_unseen_movies(ratings_matrix, 9)

# 잠재 요인 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성.
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies

,pred_score
title,
Rear Window (1954),5.704612
"South Park: Bigger, Longer and Uncut (1999)",5.451100
Rounders (1998),5.298393
Blade Runner (1982),5.244951
Roger & Me (1989),5.191962
Gattaca (1997),5.183179
Ben-Hur (1959),5.130463
Rosencrantz and Guildenstern Are Dead (1990),5.087375
"Big Lebowski, The (1998)",5.038690


##08. 파이썬 추천 시스템 패키지 - Surprise

###Surprise 패키지 소개
- 추천 시스템의 경우 상업적으로 가치가 크기 때문에 별도의 패키지로 제공될 경우 매우 활용도가 높음
  - 이에 따라 파이썬 기반의 추천 시스템 구축을 위한 전용 패키지 중의 하나인 Surprise를 소개하고자 함
- Surprize 패키지: API를 이용하에 쉽게 추천 시스템을 구축할 수 있도록 만들어짐
  - 장점
    - 다양한 추천 알고리즘을 기반으로 SVD, SVD++, NMF 기반의 잠재 요인 협업 필터링을 쉽게 적용하여 추천 시스템을 구축 가능
    - Surprise의 핵심 API는 사이킷런의 핵심 API와 유사한 API명으로 작성됨
    

In [10]:
!pip install numpy==1.26.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 144.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,

In [1]:
!pip install scikit-surprise

  Using cached scikit_surprise-1.1.4.tar.gz (154 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp312-cp312-linux_x86_64.whl size=2611211 sha256=9ba6af5b66e6069a05a1ab035af53350959f7209a2959908a1aab620533d8815
  Stored in directory: /root/.cache/pip/wheels/75/fa/bc/739bc2cb1fbaab6061854e6cfbb81a0ae52c92a502a7fa454b
Successfully built scikit-surprise


###Surprize를 이용한 추천 시스템 구축
- 추천 데이터를 학습용과 테스트용 데이터 세트로 분리한 뒤 SVD 행렬 분해를 통한 잠재 요인 협업 필터링을 수행하고자 함

In [2]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [3]:
# 추천을 위한 데이터 세트를 로딩
data = Dataset.load_builtin('ml-100k')
# 수행 시 마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [4]:
# SVD로 잠재 요인 협업 필터링을 수행
algo = SVD(random_state=0)
# fit을 이용해 학습 데이터 세트 기반으로 추천 알고리즘을 학습
algo.fit(trainset)

In [5]:
# 학습된 추천 알고리즘을 기반으로 테스트 데이터 세트에 대해 추천을 수행
predictions = algo.test(testset)
print('prediction type: ', type(predictions), 'size: ', len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type:  <class 'list'> size:  25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.5114147666251547, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.573872419581491, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.033583485472447, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.8463639495936905, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.1807542478219157, details={'was_impossible': False})]

- SVD 알고리즘 객체의 test 메서드의 호출 결과는 파이썬 리스트이고, 크기는 입력 인자 데이터 세트의 크기와 같은 25000개임
- 호출 겨로가로 반환된 리스트 객체는 25,000개의 prediction 객체를 내부에 가짐
- prediction 객체는 surprise 패키지에서 제공하는 데이터 타입이며, 개별 사용자 아이디(uid), 영화 아이디(iid), 실제 평점(r_ui) 정보에 기반해 surprize의 추천 예측 평점 데이터를 튜플 형태로 가짐

In [6]:
[(pred.uid, pred.iid, pred.est) for pred in predictions[:3]]

[('120', '282', 3.5114147666251547),
 ('882', '291', 3.573872419581491),
 ('535', '507', 4.033583485472447)]

In [7]:
# surprise 패키지의 다른 추천 예측 메서드인 predict()를 이용해 추천 예측을 진행
# 사용자 아이디, 아이템 아이디 문자열로 입력해야 함.
uid = str(196)
iid = str(302)
pred = algo.predict(uid,iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


- predict()는 개별 사용자와 아이템 정보를 입력하면 추천 예측 평점을 est로 반환

In [8]:
# 테스트 데이터 세트를 이용해 추천 예측 평점과 실제 평점과의 차이를 평가
accuracy.rmse(predictions)

RMSE: 0.9467


0.9466860806937948

- surprise패키지를 이용하면 쉽게 추천 시스템을 구현할 수 있음

###Surprise 주요 모듈 소개

**Dataset**
- surprise의 경우 사용자 아이디, 아이템 아이디, 평점 데이터가 로우 레벨로 된 데이터 세트만 적용 가능함
- 이외의 세트는 로딩에서 제외되는 특징을 가짐
- 단, 데이터 세트의 칼럼 순서가 사용자 아이디, 아이템 아이디, 평점 순으로 반드시 되어 있어야 함

**OS 파일 데이터를 Surprise 데이터 세트로 로딩**
- surprise에 OS 파일을 로딩할 때 주의할 점: 로딩되는 데이터 파일에 칼럼명을 가지는 헤더 문자열이 있어서는 안됨
- 즉, 만일 헤더 칼럼명이 존재하면, 해당 헤더를 삭제한 후 이용해야 함

In [9]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-latest-small/ratings.csv')
# ratings_noh.csv 파일로 언로드시 인덱스와 헤더를 모두 제거한 새로운 파일 생성.
ratings.to_csv('/content/drive/MyDrive/Colab Notebooks/ml-latest-small/ratings_noh.csv',
               index=False, header=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
data=Dataset.load_from_file('/content/drive/MyDrive/Colab Notebooks/ml-latest-small/ratings_noh.csv', reader=reader)

In [11]:
# SVD 행렬 분해 기법을 이용하여 추천을 예측
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행 시마다 동일한 결과를 도출하기 위해 random_state 설정
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습하고 나서 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

**판다스 DataFrame에서 Surprise 데이터 세트로 로딩**
- DataFrame 역시 사용자 아이디, 아이템 아이디, 평점 칼럼 순서를 지켜야 함

In [12]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-latest-small/ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame에서 칼럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다.
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

###Surprise 추천 알고리즘 클래스
- SVD: 행렬 분해를 통한 잠재 요인 협업 필터링을 위한 SVD 알고리즘
  - SVD 입력 파라미터
    - n_factors: 잠재 요인 K의 개수, 디폴트 100
    - n_epochs: SGD 수행 시 반복 횟수, 디폴트 20
    - biased: 베이스라인 사용자 편향 적용 여부, 디폴트 True
- KNNBasic: 최근접 이웃 협업 필터링을 위한 KNN 알고리즘
- BaselineOnly: 사용자 Bias와 아이템 Bias를 감안한 SGD 베이스라인 알고리즘.
- SVD++알고리즘의 RMSE, MAE 성적이 가장 좋으나, 상대적으로 시간이 너무 오래걸리기 때문에 조금만 큰 데이터를 돌리면 사용하기 어려워짐
- 큰 데이터인 경우 상대적으로 SVD와 k-NN이 좋을 듯

###베이스라인 평점
- 개인의 성향을 반영하여 아이템 평가에 편향성 요소를 반영하여 평점을 부과하는 것을 베이스라인 평점이라고 함
- 일반적으로 베이스라인 평점은 전체 평균 평점 + 사용자 편향 점수 + 아이템 편향 점수 공식으로 계산됨
  - 전체 평균 평점: 모든 사용자의 아이템에 대한 평점을 평균한 값
  - 사용자 편향 점수: 사용자별 아이템 평점 평균 값 - 전체 평균 평점
  - 아이템 편향 점수: 아이템별 평점 평균 값 - 전체 평균 평점

###교차 검증과 하이퍼 파라미터 튜닝
- Surprise는 교차검증, 하이퍼 파라미터 튜닝을 위하여 사이킷런과 유사한 cross_validate()와 GridSearchCV를 제공함

In [13]:
from surprise.model_selection import cross_validate

# 판다스 DataFrame에서 Surprise 데이터 세트로 데이터 로딩
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-latest-small/ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId','rating']], reader)

algo = SVD(random_state=0)
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8910  0.8666  0.8691  0.8771  0.8650  0.8737  0.0096  
MAE (testset)     0.6837  0.6645  0.6661  0.6734  0.6641  0.6704  0.0075  
Fit time          1.15    1.48    2.57    1.05    1.05    1.46    0.58    
Test time         0.07    0.37    0.07    0.16    0.07    0.15    0.11    


{'test_rmse': array([0.89097762, 0.86655054, 0.86905511, 0.87705056, 0.86499428]),
 'test_mae': array([0.68372358, 0.66454292, 0.66613767, 0.67343353, 0.66407712]),
 'fit_time': (1.1485984325408936,
  1.479276418685913,
  2.5683255195617676,
  1.0514452457427979,
  1.0460536479949951),
 'test_time': (0.07263541221618652,
  0.36634182929992676,
  0.07379937171936035,
  0.1631760597229004,
  0.07481026649475098)}

In [15]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터를 딕셔너리 형태로 지정.
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200]}

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse로 수행하도로 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8784273366544175
{'n_epochs': 20, 'n_factors': 50}


###Surprise를 이용한 개인화 영화 추천 시스템 구축
- Surprise 패키지는 간결하나 기능이 부족함
  - 이번에는 이 패키지를 이용해 같은 데이터를 학습 데이터와 테스트 데이터로 분리하지 않고 전체를 학습데이터로 사용하고자 함

In [17]:
# 다음 코드는 train_test_split()으로 분리되지 않는 데이터 세트에 fit()을 호출해 오류가 발생합니다.
data= Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algo = SVD(n_factors=50, random_state=0)
algo.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [19]:
# 이렇게 전 데이터 세트를 학습 데이터로 사용하기 위해선 DatasetAutoFolds 클래스를 이용.
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5,5))
# DatasetAutoFolds 클래스를 ratings_noh.csv 파일 기반으로 생성.
data_folds = DatasetAutoFolds(ratings_file='/content/drive/MyDrive/Colab Notebooks/ml-latest-small/ratings_noh.csv', reader=reader)

# 전체 데이터를 학습 데이터로 생성함.
trainset = data_folds.build_full_trainset()

In [20]:
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [21]:
# 영화에 대한 상세 속성 정보 DataFrame로딩
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ml-latest-small/movies.csv')

# userId=9의 movieId 데이터를 추출해 movieId=42 데이터가 있는지 확인.
movieIds = ratings[ratings['userId']==9]['movieId']

if movieIds[movieIds==42].count() ==0:
  print('사용자 아이디 9는 영화 아이디 42의 평점 없음')

print(movies[movies['movieId']==42])

사용자 아이디 9는 영화 아이디 42의 평점 없음
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [22]:
uid = str(9)
iid = str(42)

pred = algo.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [25]:
def get_unseen_surprise(ratings, movies, userId):
  # 입력값으로 들어온 userId에 해당하는 사용자가 평점을 매긴 모든 영화를 리스트로 생성
  seen_movies = ratings[ratings['userId']==userId]['movieId'].tolist()

  # 모든 영화의 movieId를 리스트로 생성.
  total_movies = movies['movieId'].tolist()

  # 모든 영화의 movieId 중 이미 평점을 매긴 영화의 movieId를 제외한 후 리스트로 생성
  unseen_movies = [movie for movie in total_movies if movie not in seen_movies]
  print('평점 매긴 영화 수:', len(seen_movies), '추천 대상 영화 수:', len(unseen_movies), '전체 영화 수:', len(total_movies))

  return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742


In [29]:
def recomm_movie_by_surprise(algo, userId, unseen_movies, top_n=10):

  # 알고리즘 객체의 predict() 메서드를 평점이 없는 영화에 반복 수행한 후 결과를 list 객체로 저장
  predictions= [algo.predict(str(userId), str(movieId)) for movieId in unseen_movies]

  # prediction list 객체는 surprise의 Predictions 객체를 원소로 가지고 있음.
  # [Prediction(uid='9', id='1', est=3.69), Prediction(uid='9', iid='2', est=2.98),,,,]

  # 이를 est 값으로 정렬하기 위해서 아래의 sortkey_est 함수를 정의함.
  # sortkey_est함수는 list 객체의 sort() 함수의 키 값으로 사용되어 정렬 수행.
  def sortkey_est(pred):
    return pred.est

  # sortkey_est() 반환값의 내림 차순으로 정렬 수행하고 top_n개의 최상위 값 추출.
  predictions.sort(key=sortkey_est, reverse=True)

  top_predictions = predictions[:top_n]

  # top_n 으로 추출된 영황의 정보 추출. 영화 아이디, 추천 예상 평점, 제목 추출
  top_movie_ids = [int(pred.iid) for pred in top_predictions]
  top_movie_rating = [pred.est for pred in top_predictions]
  top_movie_titles = movies[movies.movieId.isin(top_movie_ids)]['title']

  top_movie_preds = [(id, title, rating) for id, title, rating in zip(top_movie_ids, top_movie_titles, top_movie_rating)]

  return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomm_movie_by_surprise(algo, 9, unseen_movies, top_n=10)

print('##### Top-10 추천 영화 리스트 #####')
for top_movie in top_movie_preds:
  print(top_movie[1], ":", top_movie[2])

평점 매긴 영화 수: 46 추천 대상 영화 수: 9696 전체 영화 수: 9742
##### Top-10 추천 영화 리스트 #####
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122759
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122657
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
